In [1]:
##Actualizando...
# Parámetros de la API
# Obtener la fecha actual
import requests
from datetime import datetime, timedelta
import pandas as pd
import time

In [1]:
##Actualizando...
# Parámetros de la API
# Obtener la fecha actual
import requests
from datetime import datetime, timedelta
import pandas as pd
import time


# Parámetros de la API
base_url = "https://trading.mav-sa.com.ar/cgi-bin/wspd_cgi.sh/WService=wsbroker1/cpd-concertacion-csv_v8.r"
user_id = "avsawebservice"
password = "avsa1601"

# Obtener la fecha actual
fecha_actual = datetime.now()

# Formato de fecha para la API (DD/MM/AA)
fecha_formato_api = "%d/%m/%y"

# Crear una lista para almacenar los datos
datos = []

# Bucle para generar fechas desde el principio del año actual hasta la fecha actual
fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)  # Ajustar a la medianoche
fecha = fecha_actual
while fecha <= fecha_actual:
    # Construir la fecha en formato de texto
    fecha_texto = fecha.strftime(fecha_formato_api)

    # Construir la URL de la solicitud con la fecha actual
    url = f"{base_url}?mode=ws&an={user_id}&id=avsawebservice&password={password}&fecha=26/04/24"

    # Realizar la solicitud HTTP
    requests.packages.urllib3.disable_warnings()  # Desactivar la advertencia sobre solicitud HTTPS no verificada
    response = requests.get(url, verify=False)

    # Procesar la respuesta y almacenar los datos en la lista
    datos.append([fecha_texto] + response.text.strip().split(','))

    # Avanzar al siguiente día
    fecha += timedelta(days=1)

    # Esperar 60 segundos antes de la próxima ejecución
    time.sleep(60)

# Ahora puedes trabajar directamente con la lista de datos para realizar transformaciones
# Por ejemplo, puedes convertir la lista de datos en un DataFrame de pandas y aplicar transformaciones
df = pd.DataFrame(datos)
# Realizar transformaciones adicionales en el DataFrame si es necesario
print(df)

         0                                                  1   \
0  29/04/24  Subasta; Segmento; Moneda; C-V; Tasa; Contrapa...   

                                                  2          3        4   \
0  99;461;11:44:29;26/04/2024;29/04/2024;1615023;...  00;903485  73;4487   

     5                                                  6   \
0  61;0  00;983014;30-57612427-5;11:26:52;;EX;*GAR09070...   

                                       7   \
0  4;;;114;;;;;\r\n1061562;Avalado;$;V;48   

                                                  8          9   ...       28  \
0  99;461;11:44:29;26/04/2024;29/04/2024;1615024;...  00;903485  ...  81;1081   

     29                                                 30  \
0  83;0  00;310600;30-57612427-5;13:25:12;;EX;*ARP12060...   

                                        31  \
0  18;;;449;;;;;\r\n1061745;Avalado;$;V;50   

                                                  32         33      34    35  \
0  00;449;13:42:37;26/04/2024;29/04/

In [2]:
import csv

fecha_hoy = datetime.now().strftime("%Y-%m-%d")

# Generar el nombre del archivo CSV con la fecha de hoy
nombre_archivo = f"datos_{fecha_hoy}.csv"

# Escribir los datos en el archivo CSV
with open(nombre_archivo, "w", newline="", encoding="utf-8") as archivo:
    escritor_csv = csv.writer(archivo)
    escritor_csv.writerows(datos)

# Leer los datos del archivo CSV y crear un DataFrame
df = pd.read_csv(nombre_archivo, delimiter=';', quotechar='"', lineterminator='\n')

# Mostrar el DataFrame
print(df)

   29/04/24,"Subasta  Segmento  Moneda  C-V    Tasa   Contraparte  Hora Conc.  \
0            1061562   Avalado       $    V  48",99           461    11:44:29   
1            1061562   Avalado       $    V  48",99           461    11:44:29   
2            1061562   Avalado       $    V  48",99           461    11:44:29   
3            1061562   Avalado       $    V  48",99           461    11:44:29   
4            1061744   Avalado       $    V  50",00           449    13:42:13   
5            1061745   Avalado       $    V  50",00           449    13:42:37   

   Concertación  Liquidación   Reg.Oper.  ...  Nro.Cta.Libr.  IVA Der.Mer.  \
0    26/04/2024   29/04/2024     1615023  ...            NaN        942,"4   
1    26/04/2024   29/04/2024     1615024  ...            NaN        942,"4   
2    26/04/2024   29/04/2024     1615025  ...            NaN        942,"4   
3    26/04/2024   29/04/2024     1615026  ...            NaN        942,"4   
4    26/04/2024   29/04/2024     1615270  

In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   23/04/24,"Subasta       6 non-null      int64  
 1    Segmento               6 non-null      object 
 2    Moneda                 6 non-null      object 
 3    C-V                    6 non-null      object 
 4    Tasa                   6 non-null      object 
 5    Contraparte            6 non-null      int64  
 6    Hora Conc.             6 non-null      object 
 7    Concertación           6 non-null      object 
 8    Liquidación            6 non-null      object 
 9    Reg.Oper.              6 non-null      int64  
 10   Id.Cheque              6 non-null      int64  
 11   Banco                  6 non-null      int64  
 12   Nro.Cheque             6 non-null      int64  
 13   Acreditación           6 non-null      object 
 14   Monto                  6 non-null      object

In [3]:
df = df.astype(str)


In [4]:
# Función para extraer la fecha y otros valores de la primera columna
def separar_fecha_valor(row):
    valores = row[0].split(',')
    fecha = valores[0].strip()
    otros_valores = ','.join(valores[2:]).strip()
    return pd.Series([fecha, otros_valores])

# Aplicar la función a cada fila del DataFrame y asignar los resultados a nuevas columnas
df[['Fecha', 'Subasta']] = df.apply(separar_fecha_valor, axis=1)

# Mostrar el DataFrame resultante
print(df)


  29/04/24,"Subasta  Segmento  Moneda  C-V    Tasa  Contraparte  Hora Conc.  \
0           1061562   Avalado       $    V  48",99          461    11:44:29   
1           1061562   Avalado       $    V  48",99          461    11:44:29   
2           1061562   Avalado       $    V  48",99          461    11:44:29   
3           1061562   Avalado       $    V  48",99          461    11:44:29   
4           1061744   Avalado       $    V  50",00          449    13:42:13   
5           1061745   Avalado       $    V  50",00          449    13:42:37   

   Concertación  Liquidación  Reg.Oper.  ... CUIT Librador/Deudor CUIT PyME  \
0    26/04/2024   29/04/2024    1615023  ...                  nan       nan   
1    26/04/2024   29/04/2024    1615024  ...                  nan       nan   
2    26/04/2024   29/04/2024    1615025  ...                  nan       nan   
3    26/04/2024   29/04/2024    1615026  ...                  nan       nan   
4    26/04/2024   29/04/2024    1615270  ...       

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_788\2368009279.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  valores = row[0].split(',')


In [5]:
# Supongamos que tienes un DataFrame llamado df con una columna 'fecha'

# Define una función para procesar los valores de la columna 'fecha'
def procesar_fecha(valor):
    if "/" in str(valor):
        return valor
    else:
        return None  # O cualquier otro valor que desees poner en la columna 'otros_valores'

# Aplica la función a la columna 'fecha' y guarda los resultados en una nueva columna 'fecha_procesada'
df['Fecha'] = df['Fecha'].apply(procesar_fecha)

# Crea una nueva columna 'otros_valores' con los valores que no contenían '/'
df['Subasta'] = df['Fecha'].apply(lambda x: x if "/" not in str(x) else None)

In [6]:
df['Fecha'] = df['Fecha'].fillna(df[' Concertación'])


In [7]:
df

,"29/04/24,""Subasta",Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,...,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,Fecha,Subasta
0,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615023,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,None
1,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615024,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,None
2,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615025,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,None
3,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615026,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,None
4,1061744,Avalado,$,V,"50"",00",449,13:42:13,26/04/2024,29/04/2024,1615270,...,nan,nan,449,nan,nan,nan,nan,\r,26/04/2024,None
5,1061745,Avalado,$,V,"50"",00",449,13:42:37,26/04/2024,29/04/2024,1615273,...,nan,nan,449,nan,nan,nan,nan,\r,26/04/2024,None


In [8]:
primera_columna_duplicada = df.iloc[:, 0]

# Crear la columna 'otros_valores'
df['Subasta'] = primera_columna_duplicada.apply(lambda x: x if '/' not in str(x) else None)

In [9]:
df

,"29/04/24,""Subasta",Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,...,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,Fecha,Subasta
0,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615023,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,1061562
1,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615024,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,1061562
2,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615025,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,1061562
3,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,1615026,...,nan,nan,114,nan,nan,nan,nan,\r,26/04/2024,1061562
4,1061744,Avalado,$,V,"50"",00",449,13:42:13,26/04/2024,29/04/2024,1615270,...,nan,nan,449,nan,nan,nan,nan,\r,26/04/2024,1061744
5,1061745,Avalado,$,V,"50"",00",449,13:42:37,26/04/2024,29/04/2024,1615273,...,nan,nan,449,nan,nan,nan,nan,\r,26/04/2024,1061745


In [10]:


# Eliminar las dos primeras columnas
df = df.drop(df.columns[[0]], axis=1)
df = df.drop_duplicates()

# Obtener una lista con el nombre de todas las columnas
columnas = df.columns.tolist()

# Reorganizar la lista de columnas para que "Fecha" y "Otros_Valores" estén al principio
columnas = ['Fecha', 'Subasta'] + [col for col in columnas if col not in ['Fecha', 'Subasta']]

# Reorganizar el DataFrame con las nuevas columnas
df = df[columnas]




In [11]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,...,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r
0,26/04/2024,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,...,nan,"942,""4",nan,nan,114,nan,nan,nan,nan,\r
1,26/04/2024,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,...,nan,"942,""4",nan,nan,114,nan,nan,nan,nan,\r
2,26/04/2024,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,...,nan,"942,""4",nan,nan,114,nan,nan,nan,nan,\r
3,26/04/2024,1061562,Avalado,$,V,"48"",99",461,11:44:29,26/04/2024,29/04/2024,...,nan,"942,""4",nan,nan,114,nan,nan,nan,nan,\r
4,26/04/2024,1061744,Avalado,$,V,"50"",00",449,13:42:13,26/04/2024,29/04/2024,...,nan,"227,""18",nan,nan,449,nan,nan,nan,nan,\r
5,26/04/2024,1061745,Avalado,$,V,"50"",00",449,13:42:37,26/04/2024,29/04/2024,...,nan,"129,37",nan,nan,449,nan,nan,nan,nan,\r


In [12]:
from datetime import datetime

# Convertir 'Fecha' a tipo datetime con el formato adecuado
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d/%m/%Y')

# Convertir 'Vto.' a tipo datetime con el formato adecuado
df['Vto.'] = pd.to_datetime(df['Vto.'], format='%d/%m/%Y')
df['dias_entre_fechas'] = (df['Vto.'] - df['Fecha']).dt.days

In [13]:
# Eliminar las comas de las cadenas y convertirlas a números de punto flotante
df[' Monto'] = df[' Monto'].str.split(',').str[0]
# Define una función para asignar la categoría en función de las palabras clave
def asignar_categoria(texto):
    if 'ARP' in texto:
        return 'ARPY'
    elif 'GAR' in texto:
        return 'GAR'
    elif 'MAV' in texto:
        return 'MAV'
    elif 'ACE' in texto:
        return 'ACEN'
    elif 'FDE' in texto:
        return 'FIDEM'
    elif 'NOV' in texto:
        return 'NOV'
    elif 'CRE' in texto:
        return 'CREC'
    elif 'POT' in texto:
        return 'POT'
    elif 'ACI' in texto:
        return 'ACIN'
    elif 'POT' in texto:
        return 'POT'
    elif 'BIN' in texto:
        return 'BIND'
    elif '#UGA' in texto:
        return 'GAR'
    elif '#UFA' in texto:
        return 'AVFE'
    elif '#UAR' in texto:
        return 'ARP'
    elif '#UAM' in texto:
        return 'AMER' 
    elif '#UAC' in texto:
        return 'ACIN'
    elif 'FED' in texto:
        return 'FEDE'
    elif 'FINT' in texto:
        return 'FINT'
    
    # AgregaMOS más condiciones según sea necesario
    else:
        return 'otra_categoria'

# Aplica la función a la columna 'texto' para crear la nueva columna 'SGR'
df['SGR'] = df[' Cod.Ch.'].apply(asignar_categoria)

# Muestra el DataFrame con la nueva columna
print(df)

       Fecha  Subasta  Segmento  Moneda  C-V    Tasa  Contraparte  Hora Conc.  \
0 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
1 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
2 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
3 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
4 2024-04-26  1061744   Avalado       $    V  50",00          449    13:42:13   
5 2024-04-26  1061745   Avalado       $    V  50",00          449    13:42:37   

   Concertación  Liquidación  ... CUIT Librador/Deudor CUIT PyME  \
0    26/04/2024   29/04/2024  ...                  nan       nan   
1    26/04/2024   29/04/2024  ...                  nan       nan   
2    26/04/2024   29/04/2024  ...                  nan       nan   
3    26/04/2024   29/04/2024  ...                  nan       nan   
4    26/04/2024   29/04/2024  ...                  nan       nan   
5    26/04/2024   29/04/

In [14]:
# Configurar la opción para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Ahora puedes imprimir el DataFrame y ver todas las columnas
print(df)

       Fecha  Subasta  Segmento  Moneda  C-V    Tasa  Contraparte  Hora Conc.  \
0 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
1 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
2 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
3 2024-04-26  1061562   Avalado       $    V  48",99          461    11:44:29   
4 2024-04-26  1061744   Avalado       $    V  50",00          449    13:42:13   
5 2024-04-26  1061745   Avalado       $    V  50",00          449    13:42:37   

   Concertación  Liquidación  Reg.Oper.  Id.Cheque  Banco  Nro.Cheque  \
0    26/04/2024   29/04/2024    1615023    1605612     27           0   
1    26/04/2024   29/04/2024    1615024    1605613     27           0   
2    26/04/2024   29/04/2024    1615025    1605614     27           0   
3    26/04/2024   29/04/2024    1615026    1605615     27           0   
4    26/04/2024   29/04/2024    1615270    1606015     11          

In [22]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-04-25,1061118,Avalado,$,V,"59"",98",727,12:45:46,25/04/2024,26/04/2024,1614065,1601559,20.0,0,31/05/2024,2500000,"135967,49","551,61","0,00",950026,30-60473101-8,12:36:12,nan,EX,*FDE290500017,$,1,No,No,2024-05-29,CORDOBA,330.0,No,Si,ECHEQ,CVSA,RZP90E5X16Y9EGJ,nan,nan,"115,""84",nan,nan,727,nan,nan,nan,nan,\r,34,FIDEM
1,2024-04-25,1061178,No Garantizado,U$D,C,"5"",00",427,12:26:47,25/04/2024,25/04/2024,1613990,1604661,nan,65182,13/05/2025,119000,"5932,19","29620,94","0,00",290042,30-70870027-0,12:09:59,nan,RI,#UMV090550024,$,"873,25",No,No,2025-05-09,nan,nan,No,Si,PAGARE,CVSA,000000065182,nan,nan,"6220,""4",30-70820156-8,nan,427,nan,nan,nan,nan,EL CACIQUE S.A.\r,379,otra_categoria
2,2024-04-25,1061178,No Garantizado,U$D,V,"5"",00",427,12:26:47,25/04/2024,25/04/2024,1613990,1604661,nan,65182,13/05/2025,119000,"5932,19","59241,88","0,00",290037,30-70870027-0,12:09:46,nan,RI,#UMV090550024,$,"873,25",No,No,2025-05-09,nan,nan,No,Si,PAGARE,CVSA,000000065182,nan,nan,"12440,""79",30-70820156-8,nan,427,nan,nan,nan,nan,EL CACIQUE S.A.\r,379,otra_categoria
3,2024-04-25,1061181,Garantizado,$,V,"50"",00",567,12:35:59,25/04/2024,25/04/2024,1614042,1604728,20.0,141,14/08/2024,10000000,"1319857,31","5208,09","0,00",390024,30-57612427-5,12:10:41,nan,EX,*MAV100800588,$,1,No,No,2024-08-10,CORDOBA,369.0,Si,Si,ECHEQ,CVSA,V8794X7ZQQD9PEY,2424.0,4000099.0,"1093,""7",nan,nan,567,AO,nan,nan,Certificado MiPyME,\r,107,MAV
4,2024-04-25,1061181,Garantizado,$,V,"50"",00",567,12:35:59,25/04/2024,25/04/2024,1614043,1604729,285.0,49099876,21/08/2024,20000000,"2783018,87","10330,19","0,00",390024,30-57612427-5,12:10:41,nan,EX,*MAV170800486,$,1,No,No,2024-08-17,MACRO,893.0,Si,Si,ECHEQ,CVSA,JXON1PROPRV9Z64,2400.0,9420332544.0,"2169,""34",nan,nan,567,AO,nan,nan,Certificado MiPyME,\r,114,MAV
5,2024-04-25,1061197,Avalado,$,V,"49"",99",567,12:41:13,25/04/2024,26/04/2024,1614053,1604720,7.0,0,04/06/2024,10000000,"507055,86","2468,17","0,00",290071,30-70496099-5,12:24:11,nan,EX,*FIN310500015,$,1,No,No,2024-05-31,GALICIA,111.0,No,Si,ECHEQ,CVSA,4OGNYMR71MQN0L6,nan,nan,"518,""32",nan,nan,567,nan,nan,nan,nan,\r,36,otra_categoria
6,2024-04-25,1061197,Avalado,$,V,"49"",99",567,12:41:13,25/04/2024,26/04/2024,1614054,1604721,7.0,0,04/06/2024,10000000,"507055,86","2468,17","0,00",290071,30-70496099-5,12:24:11,nan,EX,*FIN310500016,$,1,No,No,2024-05-31,GALICIA,111.0,No,Si,ECHEQ,CVSA,GYKNXD1GWDP9MPR,nan,nan,"518,""32",nan,nan,567,nan,nan,nan,nan,\r,36,otra_categoria
7,2024-04-25,1061197,Avalado,$,V,"49"",99",567,12:41:13,25/04/2024,26/04/2024,1614055,1604722,7.0,0,04/06/2024,10000000,"507055,86","2468,17","0,00",290071,30-70496099-5,12:24:11,nan,EX,*FIN310500017,$,1,No,No,2024-05-31,GALICIA,111.0,No,Si,ECHEQ,CVSA,W612EOL4JO5NGXY,nan,nan,"518,""32",nan,nan,567,nan,nan,nan,nan,\r,36,otra_categoria
8,2024-04-25,1061318,No Garantizado,$,V,"70"",00",526,14:23:31,25/04/2024,26/04/2024,1614344,1605085,11.0,791805,07/06/2024,4000000,"298174,44","1036,51","0,00",310600,30-61594737-3,14:06:30,nan,RI,*MAV050600578,$,1,No,No,2024-06-05,NACION,109.0,Si,Si,ECHEQ,CVSA,EZ8NPRMGDD424KV,5186.0,1090021417.0,"217,""67",30-70862775-1,nan,526,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,41,MAV
9,2024-04-25,1061318,No Garantizado,$,V,"70"",00",526,14:23:31,25/04/2024,26/04/2024,1614345,1605086,11.0,791806,11/06/2024,4000000,"324269,89","1127,22","0,00",310600,30-61594737-3,14:06:30,nan,RI,*MAV070600711,$,1,No,No,2024-06-07,NACION,109.0,S

In [15]:
df[' Tasa'] = df[' Tasa'].str.replace('"', '')

# Reemplazar la coma por un punto en los valores de la columna "tasa"
df[' Tasa'] = df[' Tasa'].str.replace(',', '.')

# Convertir la columna "tasa" a tipo float
df[' Tasa'] = df[' Tasa'].astype(float)

# Redondear los valores en la columna "tasa" al entero más cercano
df[' Tasa'] = df[' Tasa'].round()

In [16]:
columnas_a_ignorar = ['Fecha', 'Vto.',' Hora Conc.',' Cod.Ch.','SGR',' Moneda Liquidacion','Nombre Banco','Razón Librador/Deudor\r','Cond. Pyme', ' Banco',' C-V',' Hora Ord.',' Hora Conc.', ' Concertación', ' Liquidación', ' Reg.Oper.',' Cond.',' Banco',' Acreditación']



for columna in df.columns:
    if columna not in columnas_a_ignorar:
        # Verifica si los valores de la columna son numéricos
        if pd.api.types.is_numeric_dtype(df[columna]):
            # Convierte la columna a tipo de datos Int64
            df[columna] = df[columna].astype('Int64')

# Muestra el DataFrame con las columnas convertidas
print(df)


df[' Monto'] = df[' Monto'].astype(int)


       Fecha  Subasta  Segmento  Moneda  C-V   Tasa  Contraparte  Hora Conc.  \
0 2024-04-26  1061562   Avalado       $    V     49          461    11:44:29   
1 2024-04-26  1061562   Avalado       $    V     49          461    11:44:29   
2 2024-04-26  1061562   Avalado       $    V     49          461    11:44:29   
3 2024-04-26  1061562   Avalado       $    V     49          461    11:44:29   
4 2024-04-26  1061744   Avalado       $    V     50          449    13:42:13   
5 2024-04-26  1061745   Avalado       $    V     50          449    13:42:37   

   Concertación  Liquidación  Reg.Oper.  Id.Cheque  Banco  Nro.Cheque  \
0    26/04/2024   29/04/2024    1615023    1605612     27           0   
1    26/04/2024   29/04/2024    1615024    1605613     27           0   
2    26/04/2024   29/04/2024    1615025    1605614     27           0   
3    26/04/2024   29/04/2024    1615026    1605615     27           0   
4    26/04/2024   29/04/2024    1615270    1606015     11           0   
5

In [17]:
df = df.drop_duplicates("Subasta")


In [18]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-04-26,1061562,Avalado,$,V,49,461,11:44:29,26/04/2024,29/04/2024,1615023,1605612,27,0,12/07/2024,10000000,"903485,73","4487,61","0,00",983014,30-57612427-5,11:26:52,nan,EX,*GAR090700116,$,1,No,No,2024-07-09,SUPERVIL.,26,No,Si,ECHEQ,CVSA,6DVNOEPMXE125J8,nan,nan,"942,""4",nan,nan,114,nan,nan,nan,nan,\r,74,GAR
4,2024-04-26,1061744,Avalado,$,V,50,449,13:42:13,26/04/2024,29/04/2024,1615270,1606015,11,0,14/06/2024,3750000,"222293,81","1081,83","0,00",310600,30-57612427-5,13:25:12,nan,EX,*ARP120600095,$,1,No,No,2024-06-12,NACION,109,No,Si,ECHEQ,CVSA,RPQ2G8EDX8G2GLY,nan,nan,"227,""18",nan,nan,449,nan,nan,nan,nan,\r,47,ARPY
5,2024-04-26,1061745,Avalado,$,V,50,449,13:42:37,26/04/2024,29/04/2024,1615273,1606009,17,0,28/06/2024,1666666,"126582,23","616,03","0,00",983011,30-57612427-5,13:25:35,nan,EX,*ARP260600091,$,1,No,No,2024-06-26,FRANCES,282,No,Si,ECHEQ,CVSA,KYZ9QEG7GOQ9V50,nan,nan,"129,37",nan,nan,449,nan,nan,nan,nan,\r,61,ARPY


In [27]:
df

,Fecha,Subasta,Segmento,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR
0,2024-04-25,1061118,Avalado,$,V,60,727,12:45:46,25/04/2024,26/04/2024,1614065,1601559,20.0,0,31/05/2024,2500000,"135967,49","551,61","0,00",950026,30-60473101-8,12:36:12,nan,EX,*FDE290500017,$,1,No,No,2024-05-29,CORDOBA,330.0,No,Si,ECHEQ,CVSA,RZP90E5X16Y9EGJ,nan,nan,"115,""84",nan,nan,727,nan,nan,nan,nan,\r,34,FIDEM
1,2024-04-25,1061178,No Garantizado,U$D,C,5,427,12:26:47,25/04/2024,25/04/2024,1613990,1604661,nan,65182,13/05/2025,119000,"5932,19","29620,94","0,00",290042,30-70870027-0,12:09:59,nan,RI,#UMV090550024,$,"873,25",No,No,2025-05-09,nan,nan,No,Si,PAGARE,CVSA,000000065182,nan,nan,"6220,""4",30-70820156-8,nan,427,nan,nan,nan,nan,EL CACIQUE S.A.\r,379,otra_categoria
3,2024-04-25,1061181,Garantizado,$,V,50,567,12:35:59,25/04/2024,25/04/2024,1614042,1604728,20.0,141,14/08/2024,10000000,"1319857,31","5208,09","0,00",390024,30-57612427-5,12:10:41,nan,EX,*MAV100800588,$,1,No,No,2024-08-10,CORDOBA,369.0,Si,Si,ECHEQ,CVSA,V8794X7ZQQD9PEY,2424.0,4000099.0,"1093,""7",nan,nan,567,AO,nan,nan,Certificado MiPyME,\r,107,MAV
5,2024-04-25,1061197,Avalado,$,V,50,567,12:41:13,25/04/2024,26/04/2024,1614053,1604720,7.0,0,04/06/2024,10000000,"507055,86","2468,17","0,00",290071,30-70496099-5,12:24:11,nan,EX,*FIN310500015,$,1,No,No,2024-05-31,GALICIA,111.0,No,Si,ECHEQ,CVSA,4OGNYMR71MQN0L6,nan,nan,"518,""32",nan,nan,567,nan,nan,nan,nan,\r,36,otra_categoria
8,2024-04-25,1061318,No Garantizado,$,V,70,526,14:23:31,25/04/2024,26/04/2024,1614344,1605085,11.0,791805,07/06/2024,4000000,"298174,44","1036,51","0,00",310600,30-61594737-3,14:06:30,nan,RI,*MAV050600578,$,1,No,No,2024-06-05,NACION,109.0,Si,Si,ECHEQ,CVSA,EZ8NPRMGDD424KV,5186.0,1090021417.0,"217,""67",30-70862775-1,nan,526,AO,nan,nan,Certificado MiPyME,SANTIAGO CEREALES S R L\r,41,MAV
21,2024-04-25,1061375,Avalado,$,V,48,461,16:28:09,25/04/2024,26/04/2024,1614879,1605280,7.0,0,20/05/2024,1212224,"37089,21","188,02","0,00",390041,30-70496099-5,16:26:08,nan,EX,*GAR160500207,$,1,No,No,2024-05-16,GALICIA,109.0,No,Si,ECHEQ,CVSA,G0QNDVKL4YDNEXY,nan,nan,"39,""48",nan,nan,114,nan,nan,nan,nan,\r,21,GAR
23,2024-04-25,1061436,Avalado,$,V,55,406,16:06:25,25/04/2024,26/04/2024,1614823,1605422,15.0,0,26/06/2024,10000000,"841801,53","3724,33","0,00",983014,30-51642044-4,16:04:23,nan,EX,*ARP230600111,$,1,No,No,2024-06-23,ICBC,5.0,No,Si,ECHEQ,CVSA,6DVNOEP686G25J8,nan,nan,"782,""11",nan,nan,406,nan,nan,nan,nan,\r,59,ARPY


In [19]:
def asignar_periodo(dias_entre_fechas):
    if dias_entre_fechas < 60:
        return '0 a 60'
    elif 60 <= dias_entre_fechas < 90:
        return '60 a 90'
    elif 90 <= dias_entre_fechas < 120:
        return '90 a 120'
    elif 120 <= dias_entre_fechas < 180:
        return '120 a 180'
    elif 180 <= dias_entre_fechas < 210:
        return '180 a 210'
    elif 210 <= dias_entre_fechas < 240:
        return '210 a 240'
    elif 240 <= dias_entre_fechas < 300:
        return '240 a 300'
    else:
        return 'Más de 300'

# Aplicaa mosla función a la columna 'dias_entre_fechas' para crear la nueva columna 'periodo'
df['Periodo'] = df['dias_entre_fechas'].apply(asignar_periodo)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_788\929678408.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Periodo'] = df['dias_entre_fechas'].apply(asignar_periodo)


In [20]:
## Reordenar las columnas
# Para obtener el listado de columnas
columnas = df.columns
print("Listado de columnas:")
print(columnas)



Listado de columnas:
Index(['Fecha', 'Subasta', ' Segmento', ' Moneda', ' C-V', ' Tasa',
       ' Contraparte', ' Hora Conc.', ' Concertación', ' Liquidación',
       ' Reg.Oper.', ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación',
       ' Monto', ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente',
       ' CUIT Comprador', ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.',
       ' Moneda Liquidacion', ' Tipo de Cambio', ' Sin Recurso',
       ' No a la Orden', 'Vto.', 'Nombre Banco', 'N. Sucursal', 'Pyme',
       'Primera.Neg', 'Tipo Instrumento', 'Custodio/Registro', 'Echeqid',
       'Plaza Cheque', 'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor',
       'CUIT PyME', 'Liquidador Compra', 'Caracter', 'CUIT Benef.',
       'Razón Benef.', 'Cond. Pyme', 'Razón Librador/Deudor\r',
       'dias_entre_fechas', 'SGR', 'Periodo'],
      dtype='object')


In [21]:
# Para reordenar las columnas, puedes proporcionar un nuevo orden de columnas
# Por ejemplo, si deseas reorganizar las columnas en un nuevo orden específico
nuevo_orden = ['Fecha', 'Subasta',' Moneda' ,' C-V', ' Tasa',
       ' Contraparte', ' Hora Conc.', ' Concertación', ' Liquidación',
       ' Reg.Oper.', ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación',
       ' Monto', ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente',
       ' CUIT Comprador', ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.',
       ' Moneda Liquidacion', ' Tipo de Cambio', ' Sin Recurso',
       ' No a la Orden', 'Vto.', 'Nombre Banco', 'N. Sucursal', 'Pyme',
       'Primera.Neg', 'Tipo Instrumento', 'Custodio/Registro', 'Echeqid',
       'Plaza Cheque', 'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor',
       'CUIT PyME', 'Liquidador Compra', 'Caracter', 'CUIT Benef.',
       'Razón Benef.', 'Cond. Pyme', 'Razón Librador/Deudor\r',
       'dias_entre_fechas', 'SGR',' Segmento','Periodo']  # Define el nuevo orden de las columnas

# Reordena el DataFrame según el nuevo orden de columnas
df = df[nuevo_orden]

In [22]:
df.rename(columns={' Segmento': 'Segmento'}, inplace=True)
df

,Fecha,Subasta,Moneda,C-V,Tasa,Contraparte,Hora Conc.,Concertación,Liquidación,Reg.Oper.,Id.Cheque,Banco,Nro.Cheque,Acreditación,Monto,Descuento,Der.Mer.,Der.Bol.,Comitente,CUIT Comprador,Hora Ord.,Warrant,Cond.,Cod.Ch.,Moneda Liquidacion,Tipo de Cambio,Sin Recurso,No a la Orden,Vto.,Nombre Banco,N. Sucursal,Pyme,Primera.Neg,Tipo Instrumento,Custodio/Registro,Echeqid,Plaza Cheque,Nro.Cta.Libr.,IVA Der.Mer.,CUIT Librador/Deudor,CUIT PyME,Liquidador Compra,Caracter,CUIT Benef.,Razón Benef.,Cond. Pyme,Razón Librador/Deudor\r,dias_entre_fechas,SGR,Segmento,Periodo
0,2024-04-26,1061562,$,V,49,461,11:44:29,26/04/2024,29/04/2024,1615023,1605612,27,0,12/07/2024,10000000,"903485,73","4487,61","0,00",983014,30-57612427-5,11:26:52,nan,EX,*GAR090700116,$,1,No,No,2024-07-09,SUPERVIL.,26,No,Si,ECHEQ,CVSA,6DVNOEPMXE125J8,nan,nan,"942,""4",nan,nan,114,nan,nan,nan,nan,\r,74,GAR,Avalado,60 a 90
4,2024-04-26,1061744,$,V,50,449,13:42:13,26/04/2024,29/04/2024,1615270,1606015,11,0,14/06/2024,3750000,"222293,81","1081,83","0,00",310600,30-57612427-5,13:25:12,nan,EX,*ARP120600095,$,1,No,No,2024-06-12,NACION,109,No,Si,ECHEQ,CVSA,RPQ2G8EDX8G2GLY,nan,nan,"227,""18",nan,nan,449,nan,nan,nan,nan,\r,47,ARPY,Avalado,0 a 60
5,2024-04-26,1061745,$,V,50,449,13:42:37,26/04/2024,29/04/2024,1615273,1606009,17,0,28/06/2024,1666666,"126582,23","616,03","0,00",983011,30-57612427-5,13:25:35,nan,EX,*ARP260600091,$,1,No,No,2024-06-26,FRANCES,282,No,Si,ECHEQ,CVSA,KYZ9QEG7GOQ9V50,nan,nan,"129,37",nan,nan,449,nan,nan,nan,nan,\r,61,ARPY,Avalado,60 a 90


In [49]:
df.columns

Index(['Fecha', 'Subasta', ' Moneda', ' C-V', ' Tasa', ' Contraparte',
       ' Hora Conc.', ' Concertación', ' Liquidación', ' Reg.Oper.',
       ' Id.Cheque', ' Banco', ' Nro.Cheque', ' Acreditación', ' Monto',
       ' Descuento', ' Der.Mer.', ' Der.Bol.', ' Comitente', ' CUIT Comprador',
       ' Hora Ord.', ' Warrant', ' Cond.', ' Cod.Ch.', ' Moneda Liquidacion',
       ' Tipo de Cambio', ' Sin Recurso', ' No a la Orden', 'Vto.',
       'Nombre Banco', 'N. Sucursal', 'Pyme', 'Primera.Neg',
       'Tipo Instrumento', 'Custodio/Registro', 'Echeqid', 'Plaza Cheque',
       'Nro.Cta.Libr.', 'IVA Der.Mer.', 'CUIT Librador/Deudor', 'CUIT PyME',
       'Liquidador Compra', 'Caracter', 'CUIT Benef.', 'Razón Benef.',
       'Cond. Pyme', 'Razón Librador/Deudor\r', 'dias_entre_fechas', 'SGR',
       'Segmento', 'Periodo'],
      dtype='object')

In [23]:
from sqlalchemy import create_engine

# Especificamos los detalles de la conexión a la base de datos PostgreSQL
usuario = 'postgres'
contraseña = 'Camila1995.'
host = 'localhost'
puerto = '5432'
base_de_datos = 'mav2024'

# Creamos la URL de conexión a la base de datos PostgreSQL
url_de_conexion = f'postgresql://{usuario}:{contraseña}@{host}:{puerto}/{base_de_datos}'

# Creamos una instancia del motor (engine) de SQLAlchemy
engine = create_engine(url_de_conexion)

# Guardamos el nuevo DataFrame en la tabla existente en la base de datos
nombre_tabla = 'mav2024'  # Especifica el nombre de la tabla donde deseas agregar los datos
df.to_sql(nombre_tabla, engine, if_exists='append', index=False)

3